In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

****This notebook contains EDA of train data.Here,I done some analysis on each features to understand the distribution of data.I had created some features like percentage,minutes in order to get understand the performace of each user and also difficulty level of content.Up to my knowledge I tried to explore more ways to analyse the feature to get clear picture of dataset**
 
**As the datset is too large,so I had reduced the datset by changing the datatype of feature based on its range and created a dataset which i used here.******

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_pickle('../input/riiid-reduced-dataset/reduced.pkl.gzip')
data.head()

In [ ]:
data.shape

 **Some basic eda on features**

In [ ]:
sns.countplot(x='content_type_id',hue='answered_correctly',data=data)

In [ ]:
sns.countplot(x='answered_correctly',data=data)

In [ ]:
sns.countplot(x='user_answer',data=data)

In [ ]:
sns.countplot(x='user_answer',hue='answered_correctly',data=data)

**Observation:**

1.In given data,there are more content related to questions than lecture

2.And there are more users who answered correctly for attempted questions



In [ ]:
#separating the data of users who attempted only questions
new=data[data['content_type_id']==0]
new.shape

In [ ]:
#grouuping the user_od and checking how many of them given correct answer (by using sum)
#and also counting no of questions they attempted (by using count)

g=new[['user_id','answered_correctly','prior_question_elapsed_time']].groupby('user_id').sum().reset_index()
g_=new[['user_id','answered_correctly']].groupby('user_id').count().reset_index()

#g.sort_values('answered_correctly',inplace=True,ascending=False)
g.head()

In [ ]:
#now adding count column to main datagrame i.e g 
#and also finding the succes ratio of each user i.e no of crct answers/total no of question attempted

g['total']=g_['answered_correctly']
g['per']=np.round((g['answered_correctly']/g['total'])*100)
sort=g.sort_values('per',ascending=False)
sort.head()

Above dataframe tells the user's no of correct answers to his total attempted question and also the percentage of his performance

In [ ]:
def percentage_answer(type):
    
    plt.figure(figsize=(15,5))
    plt.xlabel('each users performance on their examination in terms of percentage')
    plt.ylabel('no of students')
    if type=='desc':
        per=g['per'].value_counts().sort_index(ascending=False)
        sns.barplot(x=per.index[:25],y=per.values[:25].astype('int'),order=per.index[:25].astype('int'))
        plt.title('top percentage of students who perfomed very well in exam')

        
    elif type=='asc':
        per=g['per'].value_counts().sort_index(ascending=True)
        sns.barplot(x=per.index[:25],y=per.values[:25].astype('int'),order=per.index[:25].astype('int'))  
        plt.title('least percentage of students who performed bad in exam')
    elif type=='top':
        per=g['per'].value_counts()
        sns.barplot(x=per.index[:25],y=per.values[:25].astype('int'),order=per.index[:25].astype('int'))
        plt.title('highest percentage of students who answered correctly')


In [ ]:
percentage_answer('asc')
print()
percentage_answer('desc')
percentage_answer('top')

#above plot is about the distribution of students/users falling on each percentage i.e performance

There are no higher number of users who got 100 percentage in exams ,so only few users falled below 90 percentage in exam

Coming to poor performance,only few users got very less percentage

And in whole examination,lot of users got only 50 percent sucess rate and almost lot of users fallen in between 40 to 60 percentage


In [ ]:
#checking the no of users falling in percentage



plt.figure(figsize=(20,8))
plt.ticklabel_format(style='plain', axis='y')
sns.countplot(x='per',data=g,order=g['per'].value_counts().sort_index().index)
plt.title('distribution of performace of users in terms of percentage')
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.ticklabel_format(style='plain', axis='y')
plt.ticklabel_format(style='plain', axis='y')
sns.distplot(g['per'].values,color='black',hist=False)
plt.xticks(rotation=90)
plt.show()

Interestingly,the performace of all users has gaussian distribution i.e lot of users fallen in atmost 40 to 80 percentage
and very few users had performed worst and viceversa



In [ ]:
plt.figure(figsize=(35,8))
per=g['per'].value_counts()
sns.barplot(x=per.index.astype('int'),y=per.values,order=per.index.astype('int'))
plt.title('success percentage of students ')
plt.xlabel('success percentage i.e correctly answer to total questions attempted')
plt.ylabel('no of students')

From this plot,it clear that lot of users got 50 percentage in the exams and very least users got 99 percenatge

**Top 20 students who answered correctly**

In [ ]:

tp_20=g.sort_values('answered_correctly',ascending=False)[:25]

plt.figure(figsize=(15,5))
sns.barplot(x='user_id',y='answered_correctly',data=tp_20,order=tp_20['user_id'])
plt.title('students who given more no of correct answers')
plt.xlabel('students')
plt.ylabel('no of correct answers')
plt.xticks(rotation=90)
plt.show()

**Top 20 cotent mostly seen by students**

In [ ]:
contnt=data['content_id'].value_counts()[:25]
print('total no of unique content:',len(contnt))

plt.figure(figsize=(15,5))
sns.barplot(x=contnt.index,y=contnt.values,order=contnt.index)
plt.title('top 25 contents mostly watched by students ')
plt.xlabel('content_id')
plt.ylabel('no of students watched')
plt.xticks(rotation=90)
plt.show()

**Analysis on time taken by users**

In [ ]:
g.head()

In [ ]:
#adding mintue column to data,so that it tells the time taken by each user in minutes
g['mintues']=np.round(g['prior_question_elapsed_time']/60000)
g.head()

In [ ]:
plt.figure(figsize=(15,6))
sns.distplot(g['mintues'].values,hist=False)
plt.ticklabel_format(style='plain', axis='y')
plt.ticklabel_format(style='plain', axis='x')
plt.title('no of minutes spent by each users')
plt.xlabel('time')
plt.show()

**Observation:**

The distribution of  time spent by all users seems to be right skewed i.e there are only few users who took more time than others
so it is skewed and there are lot os users who took almost same time

But Above chart is not showing clear picture,so trying to take only less data to see the clear distribution of time

In [ ]:
g['mintues'].mean(),g['mintues'].min(),g['mintues'].max(),np.percentile(g['mintues'].values,99),len(np.unique(g['mintues'].values))

From above code,it is clear that 99 of users took time below 1519 but in data max time took by user is 17383,so there are few users who  took lot of time than ususal which seems to be rare

In [ ]:
# below function takes data based on given condition 
def dist_of_minutes(index,type):
    if type=='most':
        user_tim=g['mintues'].value_counts()[:index]  ##taking data in descending order of time spent by all users i.e gives mintues
                                                        #where most of users spent
        small_d=g['mintues'][g['mintues'].isin(user_tim.index)]
        plt.figure(figsize=(15,6.5))
        sns.distplot(small_d.values)
        plt.ticklabel_format(style='plain', axis='y')
        plt.ticklabel_format(style='plain', axis='x')
        plt.title('no of minutes spent by most of users')
        plt.show()
        
    elif type=='rare':
        user_tim=g['mintues'].value_counts().sort_values() #taking data in ascedning order of time spent by all users i.e gives mintues
                                                            #where most of users spent very less
        small_d=g['mintues'][g['mintues'].isin(user_tim.index[:index])]
        plt.figure(figsize=(15,6.5))
        sns.distplot(small_d.values)
        plt.ticklabel_format(style='plain', axis='y')
        plt.ticklabel_format(style='plain', axis='x')
        plt.title('no of minutes spent by very few users')
        plt.show()

In [ ]:
dist_of_minutes(1000,type='most')

In [ ]:
dist_of_minutes(100,type='most')

From above two graphs,it is clear that in most of users spent more time in between 0 to 20 mins 

And the very few users falling into the bin as the time increases..in precise lot of users spending very less time and very few users 
spending lot of time



In [ ]:
dist_of_minutes(100,type='rare')

And there are very few users who spending lot of time as shown above

In [ ]:
g.head()

In [ ]:
val=g['mintues'].value_counts()
plt.figure(figsize=(14,5))
sns.barplot(x=val.index[:35],y=val.values[:35],order=val.index[:35])
plt.title('Time spent by most of users')
plt.xlabel('time ')
plt.ylabel('No of users')
plt.show()

**Observation:**

Most of users spent 10 minutes in exam/content and almost lot of users spent time in between 10 to 25 minutes

In [ ]:
plt.figure(figsize=(30,8))
sns.barplot(x=val.index[-60:],y=val.values[-60:],order=val.index[-60:])
plt.xticks(rotation=90)
plt.title('Time spent by very less no of users')
plt.xlabel('time ')
plt.ylabel('No of users')
plt.show()


**Observation:**

Very few users spent more time and that too every indiviudal user had unique time which is more

**Most time spent by all users and their performance**

In [ ]:

min_10=g[g['mintues']==10.0]

plt.figure(figsize=(25,8))
sns.countplot(x='per',data=min_10,order=min_10['per'].value_counts().index)
plt.xticks(rotation=90)
plt.show()

**Observation:**

As 10.00 mintues is time spent by most of users in examination,so above plot gives the percentage result of those users who spent 10mins in exam

So,it is clear that most of users who spent 10min had got only 50 percentage followed by 33 percent i.e not good performance

**time spend by user who answered very correctly i.e top 20 users who given more correct answer**

In [ ]:
def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i, y[i], y[i], ha = 'center')

In [ ]:
sort_ans=g.sort_values('answered_correctly',ascending=False)[:20]
sort_ans.shape

plt.figure(figsize=(12,6))
sns.barplot(x='answered_correctly',y='mintues',data=sort_ans,order=sort_ans['answered_correctly'])

plt.xticks(range(len(sort_ans)),sort_ans['user_id'])
plt.xticks(rotation=90)
addlabels(sort_ans['user_id'].values,sort_ans['mintues'].values)



plt.show()


**Average time spend by user who got top percentage in exam**

In [ ]:
grp_per=g.groupby(['per']).sum().round().reset_index().sort_values('per')

plt.figure(figsize=(40,15))
sns.barplot(x='per',y='mintues',data=grp_per,order=grp_per['per'])
plt.xticks(rotation=90)
plt.show()

#addlabels(grp_per['per'].values,grp_per['mintues'].values)

**Observation:**

Suprisingly, based on performace percentage the time spent by each set of user for particualar percentage follows gaussain distribution
it means that lot of time is spent by users who had mid range percentage i.e 55 to 80

And users who had 69 percentage spent more time on content

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


g[['norm_ans','norm_total','norm_min']]=scaler.fit_transform(g[['answered_correctly','total','mintues']])
g.head()

In [ ]:
s=g.sort_values('total',ascending=True)  #sorting the data based on total question attempoted by each user
s.head()

In [ ]:

plt.figure(figsize=(25,8))


plt.plot(range(len(s)),s['mintues'].values,label='min',color='orange')

plt.plot(range(len(s)),s['total'].values,label='total',color='black')
plt.plot(range(len(s)),s['answered_correctly'].values,label='answered_corectly',color='red')

plt.title('distribution of time and correct_answers wrt total questions ')
plt.xlabel('No of Total questions')
plt.legend()
plt.show()

**Observation:**

It seems like a pareto distribution i.e lot of users who attempted very less questions and very less users attempted more questions
and times also following the same pattern of total questions,as questions increasing time also increasing 

In [ ]:
plt.figure(figsize=(15,8))
sns.distplot(s['total'].values,hist=False,label='total')
sns.distplot(s['mintues'].values,hist=False,label='minutes')
sns.distplot(s['answered_correctly'].values,hist=False,label='answered corectly')
plt.legend()
plt.show()

**Observation:**

And three features follows the same dsitribution i.e total questions and corect answers and time are related to each other

In [ ]:
grp_per.head()

In [ ]:
plt.figure(figsize=(30,12))
plt.ticklabel_format(style='plain', axis='y')
sns.barplot(x='per',y='total',data=grp_per,color='red')
sns.barplot(x='per',y='answered_correctly',data=grp_per,color='blue')
sns.barplot(x='per',y='mintues',data=grp_per,color='black')
plt.xticks(rotation=90)
plt.show()

**Observation:**

Total Question,answered_corecctly and time follows the gaussain distribution when percantge of users is considered

As we know percentage/performance of users following the gausaiin distribution,so the total question attempted by each percentage users also follwong the gaussian vibes and interestly out of total questions ,the correct answers are also folling gaussain

And as we come to know that time is dependnt on total questions,so based on questions distribution ,time distribution is also matching 

In final,there lot of questions/time spent by the users who got percentage of 50 to 80 range and rest of them have very less contribution of questions ,so time also low

**Finding the level of content i.e easy ,medium and hard**

There may be case like for some x content, 1000 users attempted and only 20 given correct answers
so that we get to know that x conetnt is very tough to students

In [ ]:
new.head()

In [ ]:
cont=new[['content_id','answered_correctly','prior_question_elapsed_time']].groupby('content_id').sum().reset_index()
cont_=new[['content_id','answered_correctly']].groupby('content_id').count().reset_index()
cont['tot_ques']=cont_['answered_correctly'].values
cont['min']=np.round(cont['prior_question_elapsed_time']/60000)
cont['per']=np.round((cont['answered_correctly']/cont['tot_ques'])*100)
cont.head()

In [ ]:
plt.figure(figsize=(25,10))
plt.ticklabel_format(style='plain', axis='y')
sns.barplot(x=cont['per'].value_counts().index,y=cont['per'].value_counts().values)
plt.xticks(rotation=90)
plt.show()

**Observation:**

Ther percentage of answer question with respect to each contents also follows slightly gaussian distributiion

In [ ]:

cont['diff']=cont['tot_ques']-cont['answered_correctly']
new_con=cont[cont['diff']>0]
new_con.head()

**contents/questions that are very tough i.e among all users who attempted ,only few users given correct answer**

In [ ]:
plt.figure(figsize=(25,8))
tough=new_con.sort_values('diff',ascending=False)[:100]  #more the diff btn total question attempted and correctly asnweres
                                                        #it means that very less users had answered it out of total users attemoted
sns.barplot(x='content_id',y='diff',data=tough,order=tough['content_id'])
plt.title('Content id Which attempted by users but only few users given correct answers ot of total users')
plt.xticks(rotation=90)
plt.show()

**Observation:**


Above graphs shows some content id which seems very tough to users i.e. only few user given correct answers out of total users attempted.

Graphs is plotted by taking difference between total question attempted and correctly answered out of it for each content,so it may be case that more the diff more the tough the content was.




**Contents which recevied lot of correct answers irrespective of total attempted on it (purely focusing on correct answers)**


In [ ]:
plt.figure(figsize=(25,8))
easy=new_con.sort_values('answered_correctly',ascending=False)[:100]
sns.barplot(x='content_id',y='answered_correctly',data=easy,order=easy['content_id'])

plt.title('Conetnts which recevied lot of correct answers')
plt.xticks(rotation=90)
plt.show()

**Contents which out of all users who attempted this,every one given correct answers(100% success ratio of that content)**

In [ ]:
zero=cont[cont['diff']==0]
zero.head()

In [ ]:
print('content with 100% success ratio:')
      
print(zero['content_id'].values)

Above content seems easy so that each user who attempted given correct answer

**About lecture content**

In [ ]:
lec=data[data['content_type_id']==1]
lec.head()

In [ ]:
lec.shape

**User who watched more lectures**

In [ ]:
def count(length,feature,typee,title):
    if typee=='top':
        val=lec[feature].value_counts()[:length]
        plt.figure(figsize=(8,5))
        sns.barplot(x=val.index,y=val.values,order=val.index)
        plt.xlabel(feature)
        plt.title(title)
        plt.ylabel('count')
        plt.xticks(rotation=90)
        plt.show()
    elif typee=='least':
        val=lec[feature].value_counts().sort_values()[:length]
        plt.figure(figsize=(8,5))
        sns.barplot(x=val.index,y=val.values,order=val.index)
        plt.xlabel(feature)
        plt.title(title)
        plt.ylabel('count')
        plt.xticks(rotation=90)
        plt.show()      

In [ ]:
count(25,'user_id',typee='top',title='Top 25 users who seen more lectures')

In [ ]:
count(35,'user_id',typee='least',title='least 35 users who seen more lectures')

In [ ]:
count(25,'content_id',typee='top',title='Top 25 contents moslty viewed')

In [ ]:
count(25,'content_id',typee='least',title='least 25 contents rarely viewed')

In [ ]:
#total no of uniqye contents

length=len(lec['content_id'].value_counts())
length

**lets check question and lecture dataset******

In [ ]:
lecture=pd.read_csv('../input/riiid-test-answer-prediction/lectures.csv')
lecture.head()

In [ ]:
lecture.shape

**Exploring the lecture datset**

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='tag',data=lecture,order=lecture['tag'].value_counts().iloc[:25].index)

plt.title('tags present in data')
plt.xticks(rotation=90)
plt.show()  

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='type_of',data=lecture,order=lecture['type_of'].value_counts().index)
plt.title('typpes of lectures')
plt.xticks(rotation=90)
plt.show()  

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='part',data=lecture,order=lecture['part'].value_counts().index)
plt.title('typpes of parts present in data')
plt.xticks(rotation=90)
plt.show()  

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='part',data=lecture,hue='type_of',order=lecture['part'].value_counts().index)
plt.title('typpes of lectures')
plt.xticks(rotation=90)
plt.show()  

**Observation:**

Lectures are more related to concepts and soliving questions

And in each part,most of content is related concept and solving questions but except part 2 and part 1 every part has more concept realted lectures than solving questions

**Understanding the questions data**

In [ ]:
question=pd.read_csv('../input/riiid-test-answer-prediction/questions.csv')
question.head()

In [ ]:
question.shape

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='part',data=question,order=question['part'].value_counts().index)
plt.title('typs of parts in questions')
plt.xticks(rotation=90)
plt.show()  

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='tags',data=question,order=question['tags'].value_counts().iloc[:25].index)
plt.title('most used tags in questions')
plt.xticks(rotation=90)
plt.show()  